In [3]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from typing import Optional

class PhysicalAddress(BaseModel):
  name: str = Field(..., description="Name of the place")
  city_name: Optional[str] = Field(None, description="City Name, or Nearest City")

class Feelings(BaseModel):
  happy: bool = Field(..., description="Tone of message is happy, or brings back happy memories")
  sincere: bool = Field(..., description="Tone of message is sincere")
  sad: bool = Field(..., description="Tone of message is sad, or brings back sad memories")

human_with_address = ChatOpenAI(model="gpt-4o").with_structured_output(PhysicalAddress)
human_with_feelings = ChatOpenAI(model="gpt-4o").with_structured_output(Feelings)

In [4]:
human_with_address.invoke("It's quiet here in Joshua Tree.  The days have 36 hours.")

PhysicalAddress(name='Joshua Tree National Park', city_name='Joshua Tree')

In [5]:
human_with_feelings.invoke("It's quiet here in Joshua Tree.  The days have 36 hours.")

Feelings(happy=True, sincere=True, sad=False)

In [6]:
class HumanSoFar(BaseModel):
  feelings: Feelings
  address: PhysicalAddress
  
super_realistic_human = ChatOpenAI(model="gpt-4o").with_structured_output(HumanSoFar)

In [7]:
super_realistic_human.invoke("It's quiet here in Joshua Tree.  The days have 36 hours.")

HumanSoFar(feelings=Feelings(happy=False, sincere=False, sad=False), address=PhysicalAddress(name='Joshua Tree', city_name=None))

In [8]:
class RawHuman(BaseModel):
    human: HumanSoFar
    invoked_with: str = Field(..., description="Text given to the human to process")

In [9]:
raw_human = ChatOpenAI(model="gpt-4o").with_structured_output(RawHuman)

In [10]:
raw_human.invoke("It's quiet here in Joshua Tree.  The days have 36 hours.")

RawHuman(human=HumanSoFar(feelings=Feelings(happy=False, sincere=True, sad=False), address=PhysicalAddress(name='Joshua Tree', city_name=None)), invoked_with="It's quiet here in Joshua Tree. The days have 36 hours.")

In [11]:
raw_human.invoke("How was the climbing?  Did you guys have fun?  Oh, you had place all to yourself -- that's awesome.  That was at Sleepy Cove, right?   Next to Barker Dam.")

RawHuman(human=HumanSoFar(feelings=Feelings(happy=True, sincere=True, sad=False), address=PhysicalAddress(name='Sleepy Cove', city_name=None)), invoked_with="How was the climbing? Did you guys have fun? Oh, you had the place all to yourself -- that's awesome. That was at Sleepy Cove, right? Next to Barker Dam.")